In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn

from UDA_pytorch_utils import UDA_LSTMforSequential, UDA_pytorch_classifier_fit, \
    UDA_plot_train_val_accuracy_vs_epoch, UDA_pytorch_classifier_predict, \
    UDA_compute_accuracy

ModuleNotFoundError: No module named 'UDA_pytorch_utils'

In [43]:
torch.manual_seed(0)

In [13]:
import csv

with open('india-news-headlines-1.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    headlines1 = []
    for row in readCSV:
        headline = row[2]
        headlines1.append(headline)

In [26]:
from torchnlp.datasets import imdb_dataset
torch.manual_seed(0)

In [27]:
train_dataset = imdb_dataset(train=True)
len(train_dataset)

aclImdb_v1.tar.gz: 84.1MB [00:25, 3.26MB/s]                              


25000

In [28]:
proper_train_size = int(len(train_dataset) * 0.8)
val_size = len(train_dataset) - proper_train_size

In [29]:
proper_train_dataset, val_dataset = torch.utils.data.random_split(train_dataset,
                                                                  [proper_train_size,
                                                                   val_size])

In [30]:
proper_train_dataset[1]

{'text': '"Heartland" is a wonderful depiction of what it was really like to live on the frontier. The hard work and individual strength that were needed to survive the hardships of the climate and the lack of medical care are blended with the camaraderie and the interdependence of the settlers. The drama was especially meaningful because the story is based on the diaries of real people whose descendants still live there. It was also nice to see the west inhabited by real people. No one was glamorous or looked as if they had just spent a session with the makeup or costume department. Conchatta Ferrell is just wonderful. She is an example of the strong, persevering people who came to Wyoming in the early 20th century and let no hardship stand in their way of a new life in a new land.',
 'sentiment': 'pos'}

In [31]:
from torchnlp.encoders.text import SpacyEncoder

In [33]:
encoder = SpacyEncoder([data['text'] for data in proper_train_dataset])
proper_train_encoded = [encoder.encode(data['text']) for data in proper_train_dataset]

In [34]:
proper_train_labels = torch.tensor([int(data['sentiment'] == 'pos') for data in proper_train_dataset])

In [35]:
val_encoded = [encoder.encode(data['text']) for data in val_dataset]
val_labels = torch.tensor([int(data['sentiment'] == 'pos') for data in val_dataset])

In [36]:
proper_train_dataset_encoded = list(zip(proper_train_encoded, proper_train_labels))
val_dataset_encoded = list(zip(val_encoded, val_labels))

In [37]:
from torchnlp.word_to_vector import GloVe
pretrained_embedding = GloVe(name='6B', dim=100)

glove.6B.zip: 862MB [06:49, 2.11MB/s]                               
100%|██████████| 400000/400000 [00:33<00:00, 12050.80it/s]


In [45]:
embedding_weights = torch.Tensor(encoder.vocab_size, pretrained_embedding.dim)
for i, token in enumerate(encoder.vocab):
    embedding_weights[i] = pretrained_embedding[token]

In [46]:
simple_lstm_model = nn.Sequential(nn.Embedding.from_pretrained(embedding_weights),
                                  UDA_LSTMforSequential(100, 64),
                                  nn.Linear(64, 2))

In [47]:
num_epochs = 10  # during optimization, how many times we look at training data
batch_size = 64  # during optimization, how many training data to use at each step
learning_rate = 0.01  # during optimization, how much we nudge our solution at each step

train_accuracies, val_accuracies = \
    UDA_pytorch_classifier_fit(simple_lstm_model,
                               torch.optim.Adam(simple_lstm_model.parameters(),
                                                lr=learning_rate),
                               nn.CrossEntropyLoss(),  # includes softmax
                               proper_train_dataset_encoded, val_dataset_encoded,
                               num_epochs, batch_size,
                               sequence=True)

Epoch 1 [==================================================] 20000/20000
  Train accuracy: 0.7698
  Validation accuracy: 0.7694
Epoch 2 [==================================================] 20000/20000
  Train accuracy: 0.8174
  Validation accuracy: 0.8056
Epoch 3 [==================================================] 20000/20000
  Train accuracy: 0.8657
  Validation accuracy: 0.8486
Epoch 4 [==================================================] 20000/20000
  Train accuracy: 0.8815
  Validation accuracy: 0.8552
Epoch 5 [==================================================] 20000/20000
  Train accuracy: 0.8979
  Validation accuracy: 0.8560
Epoch 6 [==================================================] 20000/20000
  Train accuracy: 0.8307
  Validation accuracy: 0.7992
Epoch 7 [==================================================] 20000/20000
  Train accuracy: 0.9082
  Validation accuracy: 0.8572
Epoch 8 [==================================================] 20000/20000
  Train accuracy: 0.9112
  Vali

In [49]:
headlines1[1]

'win over cena satisfying but defeating undertaker bigger roman reigns'

In [51]:
# Positive is 1, Negative is 0
UDA_pytorch_classifier_predict(simple_lstm_model,
                               [encoder.encode(headlines1[1])],
                               sequence=True)

tensor([1])

In [52]:
headlines1[99]

"He is the king of 'five star' industry"

In [53]:
# Positive is 1, Negative is 0
UDA_pytorch_classifier_predict(simple_lstm_model,
                               [encoder.encode(headlines1[99])],
                               sequence=True)

tensor([1])

In [55]:
headlines1[135]

"'Dudhwa tiger died of starvation; not poisoning'"

In [54]:
# Positive is 1, Negative is 0
UDA_pytorch_classifier_predict(simple_lstm_model,
                               [encoder.encode(headlines1[135])],
                               sequence=True)

tensor([0])